In [1]:
import numpy as np
import pandas as pd
import numpy as np

# Create and fit a nearest-neighbor classifier

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('chennai-house-pricing/train.csv',parse_dates=['DATE_SALE','DATE_BUILD'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7109 entries, 0 to 7108
Data columns (total 22 columns):
PRT_ID           7109 non-null object
AREA             7109 non-null object
INT_SQFT         7109 non-null int64
DATE_SALE        7109 non-null datetime64[ns]
DIST_MAINROAD    7109 non-null int64
N_BEDROOM        7108 non-null float64
N_BATHROOM       7104 non-null float64
N_ROOM           7109 non-null int64
SALE_COND        7109 non-null object
PARK_FACIL       7109 non-null object
DATE_BUILD       7109 non-null datetime64[ns]
BUILDTYPE        7109 non-null object
UTILITY_AVAIL    7109 non-null object
STREET           7109 non-null object
MZZONE           7109 non-null object
QS_ROOMS         7109 non-null float64
QS_BATHROOM      7109 non-null float64
QS_BEDROOM       7109 non-null float64
QS_OVERALL       7061 non-null float64
REG_FEE          7109 non-null int64
COMMIS           7109 non-null int64
SALES_PRICE      7109 non-null int64
dtypes: datetime64[ns](2), float64(6), in

In [4]:
def label_sale_condition(name):
    condition=['AbNormal', 'Family', 'Partial', 'AdjLand', 'Normal Sale',
       'Ab Normal', 'Partiall', 'Adj Land', 'PartiaLl']
    for pos,item in enumerate(condition):
        if name == item:
            return pos

In [6]:
def label_zone(name):
    zone=['A', 'RH', 'RL', 'I', 'C', 'RM']
    for pos,item in enumerate(zone):
        if name == item:
            return pos

In [27]:
def label_area(name):
    areas=['Karapakkam', 'Anna Nagar', 'Adyar', 'Velachery', 'Chrompet',  'KKNagar', 'T Nagar']
    for pos,item in enumerate(areas):
        if name == item:
            return pos

In [10]:
df.PARK_FACIL.replace('Noo','No',inplace=True)
df.PARK_FACIL.replace('No',0,inplace=True)
df.PARK_FACIL.replace('Yes',1,inplace=True)
df.PARK_FACIL=df.PARK_FACIL.astype(int)

In [11]:
df.BUILDTYPE.replace('Commercial',0,inplace=True)
df.BUILDTYPE.replace('Others',1,inplace=True)
df.BUILDTYPE.replace('Other',2,inplace=True)
df.BUILDTYPE.replace('House',3,inplace=True)
df.BUILDTYPE.replace('Comercial',0,inplace=True)
df.BUILDTYPE=df.BUILDTYPE.astype(int)

In [12]:
df.UTILITY_AVAIL.replace('AllPub',0,inplace=True)
df.UTILITY_AVAIL.replace('All Pub',0,inplace=True)
df.UTILITY_AVAIL.replace('ELO',1,inplace=True)
df.UTILITY_AVAIL.replace('NoSewr ',2,inplace=True)
df.UTILITY_AVAIL.replace('NoSeWa',2,inplace=True)
df.UTILITY_AVAIL=df.UTILITY_AVAIL.astype(int)

In [13]:
df.STREET.replace('Paved',0,inplace=True)
df.STREET.replace('Gravel',1,inplace=True)
df.STREET.replace('No Access',2,inplace=True)
df.STREET.replace('Pavd',3,inplace=True)
df.STREET.replace('NoAccess',2,inplace=True)
df.STREET=df.STREET.astype(int)

In [23]:
df.AREA.replace('Karapakam','Karapakkam',inplace=True)
df.AREA.replace('Ana Nagar','Anna Nagar',inplace=True)
df.AREA.replace('Ann Nagar','Anna Nagar',inplace=True)
df.AREA.replace('Velchery','Velachery',inplace=True)
df.AREA.replace('TNagar','T Nagar',inplace=True)
df.AREA.replace('Chrompt','Chrompet',inplace=True)
df.AREA.replace('Chrmpet','Chrompet',inplace=True)
df.AREA.replace('Chormpet','Chrompet',inplace=True)
df.AREA.replace('Adyr','Adyar',inplace=True)
df.AREA.replace('KK Nagar','KKNagar',inplace=True)

In [28]:
df.loc[:,'AREA_LABEL']=df.AREA.apply(label_area)
df.loc[:,'COND_LABEL']=df.SALE_COND.apply(label_sale_condition)
df.loc[:,'ZONE_LABEL']=df.MZZONE.apply(label_zone)

In [40]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7056 entries, 3548 to 1288
Data columns (total 25 columns):
PRT_ID           7056 non-null object
AREA             7056 non-null object
INT_SQFT         7056 non-null int64
DATE_SALE        7056 non-null datetime64[ns]
DIST_MAINROAD    7056 non-null int64
N_BEDROOM        7056 non-null float64
N_BATHROOM       7056 non-null float64
N_ROOM           7056 non-null int64
SALE_COND        7056 non-null object
PARK_FACIL       7056 non-null int64
DATE_BUILD       7056 non-null datetime64[ns]
BUILDTYPE        7056 non-null int64
UTILITY_AVAIL    7056 non-null int64
STREET           7056 non-null int64
MZZONE           7056 non-null object
QS_ROOMS         7056 non-null float64
QS_BATHROOM      7056 non-null float64
QS_BEDROOM       7056 non-null float64
QS_OVERALL       7056 non-null float64
REG_FEE          7056 non-null int64
COMMIS           7056 non-null int64
SALES_PRICE      7056 non-null int64
AREA_LABEL       7056 non-null int64
COND_

In [41]:
df = shuffle(df)
train,test=train_test_split(df)

In [48]:
train_input=train[['INT_SQFT','DIST_MAINROAD', 'N_BEDROOM',
       'N_BATHROOM', 'N_ROOM', 'PARK_FACIL', 'DATE_BUILD',
       'BUILDTYPE', 'UTILITY_AVAIL', 'STREET', 'QS_ROOMS',
       'QS_BATHROOM', 'QS_BEDROOM', 'QS_OVERALL', 'REG_FEE', 'COMMIS',
                 'AREA_LABEL', 'COND_LABEL', 'ZONE_LABEL']]
train_output=train[['SALES_PRICE']]
test_input=test[['INT_SQFT','DIST_MAINROAD', 'N_BEDROOM',
       'N_BATHROOM', 'N_ROOM', 'PARK_FACIL', 'DATE_BUILD',
       'BUILDTYPE', 'UTILITY_AVAIL', 'STREET',  'QS_ROOMS',
       'QS_BATHROOM', 'QS_BEDROOM', 'QS_OVERALL', 'REG_FEE', 'COMMIS',
                 'AREA_LABEL', 'COND_LABEL', 'ZONE_LABEL']]
test_output=test[['SALES_PRICE']]


In [52]:
train_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5292 entries, 3006 to 3087
Data columns (total 19 columns):
INT_SQFT         5292 non-null int64
DIST_MAINROAD    5292 non-null int64
N_BEDROOM        5292 non-null float64
N_BATHROOM       5292 non-null float64
N_ROOM           5292 non-null int64
PARK_FACIL       5292 non-null int64
DATE_BUILD       5292 non-null datetime64[ns]
BUILDTYPE        5292 non-null int64
UTILITY_AVAIL    5292 non-null int64
STREET           5292 non-null int64
QS_ROOMS         5292 non-null float64
QS_BATHROOM      5292 non-null float64
QS_BEDROOM       5292 non-null float64
QS_OVERALL       5292 non-null float64
REG_FEE          5292 non-null int64
COMMIS           5292 non-null int64
AREA_LABEL       5292 non-null int64
COND_LABEL       5292 non-null int64
ZONE_LABEL       5292 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(12)
memory usage: 826.9 KB


In [49]:
knn = KNeighborsClassifier()
knn.fit(train_input,train_output) 
# KNeighborsClassifier()
accuracy=knn.score(test_input,test_output)
print(accuracy*100)

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [50]:
df.ZONE_LABEL.unique()

array([5, 2, 3, 4, 1, 0])